In [ ]:
#| echo: false
#| warning: false
#| output: false
#| label: loading data libraries and environment variables

import os
from dotenv import load_dotenv
from intake import open_catalog
import pandas as pd
import numpy as np
import geopandas as gpd
from cartopy import crs


load_dotenv()
usr=os.getenv("DB_USER")
pswd=os.getenv("DB_PWD")
host=os.getenv("DB_HOST")
port=os.getenv("DB_PORT")
home=os.getenv("HOME_PATH")
db_traitement=os.getenv("DB_WORKSPACE")
db_ref=os.getenv("DB_REF")
db_externe=os.getenv("DB_EXT")
dwh_fact_strategy=os.getenv("DWH_FACT_STRATEGY")
dwh_dim_strategy=os.getenv("DWH_DIM_STRATEGY")
commun_path = os.getenv("COMMUN_PATH")
project_dir = os.getenv("PROJECT_PATH")
data_catalog_dir = os.getenv("DATA_CATALOG_DIR")
data_output_dir = os.getenv("DATA_OUTPUT_DIR")
sig_data_path = os.getenv("SIG_DATA_PATH")
project_db_schema = os.getenv("PROJECT_DB_SCHEMA")


In [ ]:
#| echo: false
#| label: loading catalogs


crsRgnc = crs.LambertConformal(central_longitude=166,
                                standard_parallels=(-20.66666666666667, -22.33333333333333),
                                central_latitude=-21.5,
                                false_easting=400000,
                                false_northing=300000,
                                cutoff=0,
                                globe=crs.Globe(ellipse='GRS80'))

crs_proj4 = crsRgnc.proj4_init
#crsMerca = crs.GOOGLE_MERCATOR

# Données carto

cat_data_carto = open_catalog('./catalogue/carto.yaml')


nc_ = cat_data_carto.Grande_terre_et_iles


bb_ = cat_data_carto.bb_Grande_terre_et_iles

# Données feux

#cat_data_feux = open_catalog(f'{data_catalog_dir}feux_faits.yaml')
cat_data_feux = open_catalog('./catalogue/feux.yaml')

cat_data_feux_dtm = open_catalog(f'{data_catalog_dir}feux_dtm.yaml')
cat_data_feux_ref = open_catalog(f'{data_catalog_dir}data_reference_feux.yaml')

data_S2_ = cat_data_feux.sentinel_zones_brulees
data_SNPP_ = cat_data_feux.VIIRS_SNPP

#data_S2 = data_S2_.to_crs(3857)
# data_S2_filtreAnnee = data_S2[data_S2['annee'] == Annee]
# data_S2_filtreAnnee_NC = data_S2_filtreAnnee[data_S2['type_spatial'] == 'ZEE']


<!-- -->

In [ ]:
#| echo: false
#| output: false
#| label: load libraries and settings

import datetime
from IPython.display import display, Markdown
from tabulate import tabulate
import xarray as xr
from cartopy import crs
import holoviews as hv
import hvplot.pandas
from holoviews.element.tiles import tile_sources as ts
from holoviews import opts
from holoviews.operation.datashader import (
    rasterize, datashade, inspect_polygons
)
from holoviews.operation import timeseries

import colorcet as cc
import datashader as ds
import datashader.transfer_functions as tf

from bokeh.models import FixedTicker, HoverTool, DatetimeTickFormatter
from bokeh.models.formatters import DatetimeTickFormatter


from pyproj import Proj
from pyproj import Transformer

# warning extension('matplotlib') fait bugger l'affichage des plots geopandas.plot()
# tout le code est  fait pour être compatible avec bokeh.
hv.extension('bokeh')
# default value Analyses effectuées sur les données de l'année en cours si execute parameter n'est pas utilisé dans le lancement de la génération du rapport : --execute-params _variables.yml

today = datetime.date.today()

#Annee = today.year
Annee = 2020


export_file_path = './etude/part3/spatial-analyse_files/holoviz/'

In [ ]:
#| echo: false
#| output: false
#| label: Configuration des output holoviz et bokeh


## taille des plots fixés notament pour que les output png ne soit pas tronqué lors de l'utilisation de datashader

plot_width = 1200
plot_height = int(plot_width * 0.5)

plot_width_col2 = int(plot_width/2)
plot_height_col2 = int(plot_width_col2 * 0.5)

opts.defaults(
    opts.Polygons(
        xaxis=None, 
        yaxis=None,
    ),

    opts.Overlay( 
 


        #responsive=True
        )
    )
tiled_basemap = ts["CartoLight"]().opts(
    min_height=500, 
    #responsive=True, 
    xaxis=None, 
    yaxis=None)

In [ ]:
#| echo: false
#| output: false
#| label: configurer le rendu carto


styles = {'fontsize': 8}

texte_sources = "Sources: //////"
realisation = "Réalisation: //////"


# Coordonnées géographiques de la Nouvelle-Calédonie
lon, lat = 163.75, -22.4
lonN, latN = 167.1, -19.60
# Convertir en coordonnées Google Mercator (EPSG:3857)

transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
x, y = transformer.transform(lon, lat)
xN, yN = transformer.transform(lonN, latN)




def update_texte(x,y,texte, fontsize, halign):
    return hv.Text(x, y, texte, fontsize = fontsize,halign=halign)

def update_source(x,y,texte):
    x = x-50000
    y = y-50000
    return update_texte(x, y, f'Sources : {texte}', 8,'left' )

def update_realisation(x,y,texte):
    x = x-50000
    y = y-70000
    return update_texte(x, y, f'Réalisation : {texte}', 8,'left' )

# Positionnement du texte des sources
annotation_sources = update_source(x,y, texte_sources)

annotation_realisation = update_realisation(x,y, realisation)


# Création de la flèche
fleche_nord = hv.Arrow(xN, yN, 'N', '^').opts(
    text_color='black', 
    text_font_size= '10pt', 
    text_align='center', 
    text_baseline='middle', 
    text_font_style='bold', 
    text_alpha=0.5, 
    arrow_size=10, 
    arrow_color='black',
    arrow_alpha=0.5)    


# Définir les coordonnées de la barre d'échelle en coordonnées Google Mercator
# Ajustez ces valeurs en fonction de la taille souhaitée de votre barre d'échelle
x0, y0, x1, y1 = x-50000, y-500, x+50000, y+500 # 5000 mètres pour l'exemple

# Création de la barre d'échelle
echelle_barre = hv.Rectangles([(x0, y0, x1, y1)])
# Création de annotation d'échelle
texte_echelle = hv.Text(x, y-15000, '100 km')


::: {.content-visible }


In [ ]:
#| echo: false
#| output: true
#| label: create maps VIIRS

hover = HoverTool(tooltips=[("Nombre de surface brulées", "@image")])

data = data_SNPP_.read().to_crs(3857)
bb = bb_.read().to_crs(3857)

figureName_focus = "fig-maps-point-chaud-viirs-NC-focus"

figureName = "fig-maps-point-chaud-viirs-NC-annees"

annees = sorted(data.annee.unique().tolist())
# Suppression de la valeur
if Annee in annees:
    annees.remove(Annee)

annotation_sources = update_source(x,y, 'OEIL/NASA LANCE FIRMS VIIRS - SUOMI NPP')
annotation_realisation = update_realisation(x,y, 'OEIL')

overlay = hv.Polygons(data, vdims='annee')

# overlay pour cadrer sur la calédonie (BBox NC)
bb_overlay = hv.Polygons(bb).opts(alpha=0.0)

yearmap = tiled_basemap.opts(alpha=0.5) * \
         annotation_sources * \
         annotation_realisation * \
         texte_echelle * \
         echelle_barre * \
         fleche_nord * \
         bb_overlay * \
         rasterize(overlay.select(annee=Annee), 
            aggregator=ds.reductions.count('annee')
            ).opts(
                cmap='fire', 
                width=plot_width, 
                height=plot_height, 
                colorbar=True, 
                colorbar_opts={"ticker": FixedTicker(ticks=[2,4,6,8,10, 20, 40, 100])},
                color_levels=[0,2,4,6,8,10,20,40, 100,100.1],
                clim=(0, 100),
                tools=[hover]
                )

viz_yearmap = (yearmap).opts(title=f"Répartition des détéctions de point chauds en {Annee} ", toolbar=None)


allyearmap = tiled_basemap.opts(alpha=0.5) * \
         annotation_sources * \
         annotation_realisation * \
         texte_echelle * \
         echelle_barre * \
         fleche_nord * \
         bb_overlay * \
         rasterize(overlay, 
            aggregator=ds.reductions.count('annee')
            ).opts(
                cmap='fire', 
                width=plot_width, 
                height=plot_height, 
                colorbar=True, 
                colorbar_opts={"ticker": FixedTicker(ticks=[2,4,6,8,10, 20, 40, 100])},
                color_levels=[0,2,4,6,8,10,20,40, 100,100.1],
                clim=(0, 100),
                tools=[hover]
                )

viz_allyearmap = (allyearmap).opts(title=f"Répartition des détéctions de point chauds de {int(annees[0])} à {int(annees[-1])}", toolbar=None)




maps = hv.NdLayout( { 
         annee: tiled_basemap.opts(alpha=0.5) * 
         annotation_sources * 
         annotation_realisation *
         texte_echelle * 
         echelle_barre * 
         fleche_nord * 
         bb_overlay * 
         rasterize(overlay.select(annee=annee), 
            aggregator=ds.reductions.count('annee')
            ).opts(
                cmap='fire', 
                width=plot_width_col2, 
                height=plot_height_col2,
                color_levels=[0,2,4,6,8,10,20,40, 100,100.1],
                clim=(0, 100),
            )
         for annee in annees
         }).opts('Image').cols(2)

# Générer les maps par lot de 4 années
maps_batch = []
for i in range(0, len(annees), 4):
    maps = hv.NdLayout( { 
        annee: tiled_basemap.opts(alpha=0.5) * 
        annotation_sources * 
        annotation_realisation *
        texte_echelle * 
        echelle_barre * 
        fleche_nord * 
        bb_overlay * 
        rasterize(overlay.select(annee=annee), 
        aggregator=ds.reductions.count('annee')
        ).opts(
            cmap='fire', 
            width=plot_width_col2, 
            height=plot_height_col2,
            color_levels=[0,2,4,6,8,10,20,40, 100,100.1],
            clim=(0, 100),
        )
        for annee in annees[i:i+4]
        }).opts('Image').cols(2)
    maps_batch.append(maps)


# # export des figures en images pour les afficher dans le rapport 
saving = hv.save(viz_yearmap, f'{export_file_path}{figureName_focus}.png', backend='bokeh',  title=f'{figureName_focus}', fmt='png')
saving = hv.save(viz_yearmap, f'{export_file_path}{figureName_focus}.html', backend='bokeh',  title=f'{figureName_focus}', fmt='html')

saving = hv.save(viz_allyearmap, f'{export_file_path}{figureName}_AllYear.png', backend='bokeh',  title=f'{figureName}', fmt='png')
saving = hv.save(viz_allyearmap, f'{export_file_path}{figureName}_AllYear.html', backend='bokeh',  title=f'{figureName}', fmt='html')

i=0
for maps in maps_batch:
    # export des figures en images par lot de 4 années pour les afficher dans le rapport 

    i+=1

    viz_maps = (maps).opts(toolbar=None)

    saving = hv.save(viz_maps, f'{export_file_path}{figureName}_{i}.png', backend='bokeh',  title=f'{figureName}_{i}', fmt='png')
    saving = hv.save(viz_maps, f'{export_file_path}{figureName}_{i}.html', backend='bokeh',  title=f'{figureName}_{i}', fmt='html')




display(
    Markdown(
        f"[![]({export_file_path}{figureName_focus}.png){{width=100%}}]({export_file_path}{figureName_focus}.html)"

        )
)

In [ ]:
#| echo: false
#| label: output image map AllYear VIIRS
display(
    Markdown(
        f"[![]({export_file_path}{figureName}_AllYear.png){{width=100%}}]({export_file_path}{figureName}_AllYear.html)"
        )
)

##### Répartitions des détéctions années après années

In [ ]:
#| echo: false
#| label: output maps VIIRS

i=0
for maps in maps_batch:
    # export des figures en images par lot de 4 années pour les afficher dans le rapport 

    i+=1
    display(
        Markdown(
            f"[![]({export_file_path}{figureName}_{i}.png){{width=100%}}]({export_file_path}{figureName}_{i}.html)"
    )
    )

:::
